In [1]:
from pyspark.sql import SparkSession, functions
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
import pandas as pd
import numpy as np

In [2]:
spark = SparkSession.builder.getOrCreate()

In [78]:
#socketDF = spark.readStream.format("socket").option("host", "localhost").option("port",9000).load()
#socketDF.isStreaming()

In [79]:
#words = socketDF.select(explode(split(socketDF.value, ",")).alias("word"))

In [3]:
#query = words.writeStream.outputMode("append").format("console").start()
#query.awaitTermination()

In [70]:
#lines.writeStream.format("csv").option("path", "\2AMD15_Big_Data_Management\Assignments\Milestone 2").start()

In [3]:
text_file = spark.read.option("header", "true").csv("stream_10_100.txt")

In [4]:
text_file.show()

+---+---------+
|sid|       ip|
+---+---------+
| 38| 92585955|
|  3|347677824|
|  5|146418092|
| 22|326035506|
| 12|200422004|
| 17|333335701|
| 27|194721507|
| 10|304237092|
| 31|107460470|
| 55|137556175|
| 82| 80221636|
| 47|346099386|
| 78|360929635|
| 56|127955211|
| 12| 83644251|
| 26|300319132|
| 10| 88983965|
|  2|236759675|
| 90|384068272|
| 84|305948675|
+---+---------+
only showing top 20 rows



In [5]:
sid_10 = text_file.filter(text_file.sid.contains("10"))       # select all servers with sid = 10
ip_10 = sid_10.select(sid_10.ip)                              # from all servers with sid = 10 select only the column with ip

sid_20 = text_file.filter(text_file.sid.contains("20"))       # select all servers with sid = 20
ip_20 = sid_20.select(sid_20.ip)                              # from all servers with sid = 20 select only the column with ip

#print(ip_10.collect()[0].__getitem__("ip"))

sum = 0
product = 0
for i in range(0,ip_10.count()):
    product = int(ip_10.collect()[i].__getitem__("ip")) * int(ip_20.collect()[i].__getitem__("ip"))
    sum = sum + product
    print(i, "ip_10:", int(ip_10.collect()[i].__getitem__("ip")), "ip_20:", int(ip_20.collect()[i].__getitem__("ip")), "product:", product)
    print(sum)
print(sum)

0 ip_10: 304237092 ip_20: 370444111 product: 112702839079165212
112702839079165212
1 ip_10: 88983965 ip_20: 348295838 product: 30992744658237670
143695583737402882
2 ip_10: 291964630 ip_20: 264235887 product: 77147532980676810
220843116718079692
3 ip_10: 136387345 ip_20: 379284779 product: 51729644006721755
272572760724801447
4 ip_10: 435468035 ip_20: 219027090 product: 95379296494068150
367952057218869597
5 ip_10: 5359484 ip_20: 490059895 product: 2626468166294180
370578525385163777
6 ip_10: 474783721 ip_20: 55555679 product: 26376931998301559
396955457383465336
7 ip_10: 375929711 ip_20: 127983008 product: 48112615210350688
445068072593816024
8 ip_10: 271994032 ip_20: 367021245 product: 99827588257209840
544895660851025864
9 ip_10: 270718717 ip_20: 159101830 product: 43071843289952110
587967504140977974


IndexError: list index out of range

In [20]:
schema = StructType([
  StructField('0', IntegerType(), True),
  StructField('1', IntegerType(), True),
  StructField('2', IntegerType(), True),
  StructField('3', IntegerType(), True)
  ])
emptyRDD = spark.sparkContext.emptyRDD()
df = spark.createDataFrame(emptyRDD,schema)

In [21]:
df[1] = text_file.filter(text_file.sid.contains("1"))

TypeError: 'DataFrame' object does not support item assignment

In [24]:
for i in range(0,1000):
    print(text_file.collect()[i].__getitem__("sid"), text_file.collect()[i].__getitem__("ip"))
    

92585955
38
347677824
3
146418092
5
326035506
22


In [9]:
array = np.zeros((2**32,), dtype=int)
len(array)

4294967296